# DATA2060 Final Project

Model: **CART for classification**

Team Members:
- Muxin Fu
- Yixiao Zhang
- Jingmin Xu
- Mingrui Chen

## Introduction

#### 0.1 Overview  
The Classification and Regression Tree (CART) algorithm is a nonparametric supervised learning method that builds a binary decision tree for classification tasks. At each step, the algorithm selects a feature and threshold that create two child nodes with lower class impurity, using criteria such as Gini impurity or entropy. Through this recursive partitioning, CART represents the classifier as a set of piecewise-constant regions, where each leaf corresponds to a predicted class label. Because the sequence of splits directly mirrors the decision-making process, CART offers a transparent and intuitive model structure.

#### 0.2 Advantages  
CART offers several notable strengths that contribute to its widespread use as a baseline classifier. First, the model is highly interpretable: each internal node corresponds to a clear “if–then” condition based on a single feature, allowing the entire decision path to be easily traced and communicated. This transparency is particularly valuable in settings where model explanations are required.  

Second, CART is able to capture nonlinear relationships and feature interactions without relying on explicit transformations or parametric assumptions. Its recursive splitting procedure enables the model to adapt flexibly to irregular or complex decision boundaries, providing expressive power beyond that of linear models.  

Moreover, CART requires minimal preprocessing. It can accommodate both numerical and categorical variables, is robust to monotonic feature scaling, and implicitly performs feature selection by choosing splits only on informative variables. These characteristics make CART convenient to implement and reliable across a wide range of practical applications.

#### 0.3 Disadvantages  
Despite its advantages, CART also presents several limitations that must be considered. Most importantly, the model is prone to overfitting when allowed to grow without constraints. As emphasized in the bias–complexity trade-off discussed in the course reading, increasing model flexibility reduces approximation error but raises estimation error, causing deep, unpruned trees to exhibit high variance and poor generalization.  

CART also tends to be unstable: small perturbations in the training data can alter early splits, resulting in substantially different tree structures. This sensitivity undermines the model’s reliability, especially in contexts requiring stable predictions.  

Finally, because CART relies exclusively on axis-aligned splits, it may need many successive partitions to approximate diagonal or curved decision boundaries, leading to unnecessarily deep and complex trees. These shortcomings motivate the use of pruning techniques and more advanced ensemble methods, such as Random Forests and Gradient Boosting, which address variance and stability issues more effectively.


### 1. Representation

#### 1.1 **Domain Set**
We define the domain space as  

In the CART classification setting, each training example is represented as a feature vector in an $n$-dimensional real space:

$$
\mathcal{X} = \mathbb{R}^n, \qquad x_i = (x_{i1}, x_{i2}, \dots, x_{in}) \in \mathcal{X}.
$$

Each component $x_{ij}$ represents the value of feature $j$ for sample $i$.
The feature domain can include continuous or categorical variables (encoded numerically in practice).



#### 1.2 **Label Set**

For a $K$-class classification task, the label space is defined as:

$$
\mathcal{Y} = \{0, 1, \dots, K-1\}.
$$

In the binary case, this simplifies to:

$$
\mathcal{Y} = \{0, 1\}.
$$

#### 1.3 **Training Data**

We are given a labeled dataset:

$$
\mathcal{D} = \{(x_i, y_i)\}_{i=1}^{N},
\quad x_i \in \mathcal{X}, \; y_i \in \mathcal{Y}.
$$

Each pair $(x_i, y_i)$ represents one training example.
The training process recursively partitions $\mathcal{D}$ based on feature thresholds to form a binary decision tree.

#### 1.4 **Learner's Output**
Formally, the hypothesis space of CART classification is defined as the set of **binary decision trees** of depth at most $T_{\max}$:

$$
\mathcal{H} =
\{\, h : \mathcal{X} \to \mathcal{Y} \mid
h \text{ is a binary decision tree with depth } \le T_{\max} \,\}.
$$

Each decision tree $h \in \mathcal{H}$ recursively partitions the input space $\mathcal{X}$ into at most $2^{T_{\max}}$ disjoint leaves.

At prediction time, a new observation $x$ is passed through the sequence of feature tests $(x_f \le t)$ until it reaches a leaf node $i$.
Each leaf stores an empirical class probability vector

$$
p_i = (p_{i,0}, p_{i,1}, \dots, p_{i,K-1}),
$$

computed from the training samples that reached that leaf.

The predicted class label is then determined by

$$
\hat{y}(x) = \arg\max_k \, p_{i,k}.
$$

### 2. Loss

In the classification setting, losses are the **measures of impurity**.  CART minimizes impruity and the loss is defined per split. Generally speaking, **Gini** and **Entropy** are good measures.

To compute Loss, we need: 
* Impurity measure, 
* Split loss based on chosen impurity measure.

In the scikit-learn, this is determined by the parameter **criterion**: *{“gini”, “entropy”, “log_loss”}, default=”gini”* 



#### 2.1 **Impurity Function**

For a $K$-class classification problem, consider node $i$ containing a subset of samples

$$S_i = \{(x_j, y_j)\}_{j \in \mathcal{I}_i}, \qquad N_i = |S_i|.$$

The number of samples in node $i$ that belong to class $k$ is

$$n_{i,k} = \sum_{j \in \mathcal{I}_i} \mathbf{1}(y_j = k).$$

The class proportion of class $k$ in node $i$ is

$$p_{i,k} = \frac{n_{i,k}}{N_i}, \qquad k = 1, \dots, K.$$
$$\sum_{k=1}^K p_{i,k} = 1,\text{and  } p_{i,k} \ge 0 \quad \text{for } k = 1, \dots, K.$$

##### 2.1.1 **Gini**


- The Gini impurity of node $i$ is:   
$$G_i = 1 - \sum_{k=1}^K p_{i,k}^2.$$

##### 2.1.2 **Entropy**

- The entropy impurity of node $i$ is
$$H_i = - \sum_{k=1}^K p_{i,k} \log p_{i,k},$$

- And we assume $0 \log 0 = 0$.

#### 2.2 **Split Loss**

Given a candidate split $\theta$ applied at node $i$, the dataset $S_i$ is partitioned into a left subset $S_i^{\text{left}}(\theta)$ and a right subset $S_i^{\text{right}}(\theta)$:

$$
S_i^{\text{left}}(\theta) = \{(x_j, y_j) \in S_i \mid x_{j, f} \le t\},
$$

$$
S_i^{\text{right}}(\theta) = S_i \setminus S_i^{\text{left}}(\theta),
$$

where $\theta = (f, t)$ denotes the split feature index $f$ and the threshold value $t$.

Let the number of samples in the left and right subsets be

$$
N_i^{\text{left}} = |S_i^{\text{left}}(\theta)|, \qquad 
N_i^{\text{right}} = |S_i^{\text{right}}(\theta)|.
$$

Their corresponding class proportions are computed in the same way as in Section 2.1.


##### 2.2.1 **Weighted Child Impurity**

Given an impurity function $C(\cdot)$ (e.g., Gini or entropy), the **split loss** at node $i$ for candidate split $\theta$ is defined as the weighted sum of the left and right child impurities:

$$
L(S_i, \theta) 
= 
\frac{N_i^{\text{left}}}{N_i} 
\, C\!\left(S_i^{\text{left}}(\theta)\right)
\;+\;
\frac{N_i^{\text{right}}}{N_i}
\, C\!\left(S_i^{\text{right}}(\theta)\right).
$$

Here:

- $C\!\left(S_i^{\text{left}}(\theta)\right)$ is the impurity (Gini or entropy) of the left child node.
- $C\!\left(S_i^{\text{right}}(\theta)\right)$ is the impurity of the right child node.


##### 2.2.2 **Optimal Split Selection**

The optimal split parameter is chosen by minimizing the split loss:

$$
\theta^{*} = \arg\min_{\theta} \; L(S_i, \theta).
$$

And this will be futher explained in the next part, Optimizer on how to actually implement it.

- ***Reference***: scikit-learn mathematical formulation https://scikit-learn.org/stable/modules/tree.html#tree-mathematical-formulation

### 3. Optimizer

### 3.1 What is Optimized in CART

CART performs a **greedy, recursive partitioning** - at each node, it selects the best split that maximizes information gain (or equivalently minimizes impurity).

So the optimizer is essentially a **greedy search algorithm** that finds:

$$
\arg\min_{(f,t)} \; \text{Impurity}(S_{\text{left}}) + \text{Impurity}(S_{\text{right}})
$$

where $f$ is the feature and $t$ is the threshold.


#### 3.1.1 Objective Function


CART minimizes an **impurity measure** (loss function) such as:
- Gini Index:
$$ G(S) = 1 - \sum_{k=1}^{K}p_k^2 $$
- Entropy:
$$ H(S) = - \sum_{k=1}^{K}p_klog(p_k)$$

At each node:
$$
\text{Gain}(S, f, t) = \text{Impurity}(S) 
- \frac{|S_{\text{left}}|}{|S|} \, \text{Impurity}(S_{\text{left}}) 
- \frac{|S_{\text{right}}|}{|S|} \, \text{Impurity}(S_{\text{right}})
$$

The algorithm chooses the feature $f*$ and threshold $t*$ that maximize this gain.

#### 3.1.2 Pseudo-code

Intuitively, the algorithm asks: “Which feature and cutoff most cleanly separates the classes?” By evaluating all possible splits and picking the one that reduces impurity the most, CART greedily chooses the single question that best organizes the data at this point in the tree. This local optimization step is repeated recursively to grow the whole decision tree.

```python
Inputs: dataset S, feature set F, impurity measure Impurity()

best_gain ← 0  
best_feature, best_threshold ← None  

for each feature f in F:  
 for each possible threshold t in f:  
  Split S into S_left and S_right using (f, t)  
  if either split is empty: continue  
  gain ← Impurity(S) 
     - (|S_left| / |S|) * Impurity(S_left)
     - (|S_right| / |S|) * Impurity(S_right)  
  if gain > best_gain:  
   best_gain ← gain  
   best_feature ← f  
   best_threshold ← t  

return (best_feature, best_threshold)



This pseudocode describes how CART chooses the best split at a node by searching over all features and all possible thresholds. It begins by initializing best_gain and empty placeholders for the best split. For each feature, the algorithm tries every potential threshold and divides the dataset into **S_left** and **S_right**. If the split is invalid (one side is empty), it skips it. Otherwise, it computes the **impurity reduction** (gain): the impurity of the parent minus the weighted impurities of the two child subsets. If this gain is better than any previous one, the algorithm updates the best feature and threshold.

## Model

## Check Model

In [3]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# breast_cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target  # 0/1

# split train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=0,
    stratify=y
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Class counts:", np.bincount(y))

X_train shape: (398, 30)
X_test shape: (171, 30)
Class counts: [212 357]


In [ ]:
def make_cart_model():
    """
    Helper function to initialize our CART model.

    创建一个还没训练过的 CART 分类器实例。
    TODO: 把里面的参数名字、类名改成model实际用的!!!!!!!!!!!!!!!!!!!!!!
    """
    model = CARTClassifierScratch(
        max_depth=5,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=0
    )
    return model


### Test 1 — train() runs correctly

In [ ]:
# Test 1
model = make_cart_model()
model.train(X_train, y_train)

print("Test 1 passed: train() runs without error.")

### Test 2 — predict() shape & class values check

In [ ]:
# Test 2: predict() should produce outputs with correct shape and valid class values

model = make_cart_model()
model.train(X_train, y_train)

y_pred_train = model.predict(X_train)

# Check output shape
assert y_pred_train.shape == y_train.shape, \
    f"Prediction shape mismatch: y_pred shape={y_pred_train.shape}, y_train shape={y_train.shape}"

# Check value range (breast_cancer is a binary classification dataset)
unique_vals = np.unique(y_pred_train)
assert set(unique_vals).issubset({0, 1}), \
    f"Predicted values must be 0/1. Found values: {unique_vals}"

train_acc = accuracy_score(y_train, y_pred_train)
print(f"Test 2 passed: predict() shape & value checks passed. Train accuracy = {train_acc:.3f}")


### Test 3 — loss() returns a finite scalar

In [ ]:
# Test 3: loss() should return a finite scalar value

model = make_cart_model()
model.train(X_train, y_train)

train_loss = model.loss(X_train, y_train)

assert np.isscalar(train_loss), "loss() should return a scalar value."
assert np.isfinite(train_loss), "loss() should not return NaN or infinity."

print(f"Test 3 passed: loss() returns a valid finite scalar. Train loss = {train_loss:.6f}")


Our loss function is defined as the misclassification error rate, therefore it should be a scalar between 0 and 1.

### Test 4: Edge case testing

In [ ]:
# A small toy dataset for edge case testing
X_toy = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0],
])
y_toy = np.array([0, 0, 1, 1])

print("X_toy:\n", X_toy)
print("y_toy:", y_toy)


#### Test 4.1 — Edge case: all labels are the same (0)

In [ ]:
# Test 4.1: all labels are zero (only one class present)

model_zero = make_cart_model()

y_all_zero = np.zeros_like(y_toy)
model_zero.train(X_toy, y_all_zero)

y_pred_zero = model_zero.predict(X_toy)
loss_zero = model_zero.loss(X_toy, y_all_zero)

assert y_pred_zero.shape == y_all_zero.shape
assert np.isfinite(loss_zero)

print("Test 4.1 passed: all-zero labels edge case handled correctly.")
print("Predicted labels:", y_pred_zero)
print("Loss on all-zero labels:", loss_zero)


#### Test 4.2 — Edge case: single feature only

In [ ]:
# Test 4.2: dataset contains only one feature

model_single = make_cart_model()

X_single = X_toy[:, :1]  # Use only the first feature
model_single.train(X_single, y_toy)

y_pred_single = model_single.predict(X_single)
assert y_pred_single.shape == y_toy.shape

print("Test 4.2 passed: single-feature edge case handled correctly.")
print("Predicted labels:", y_pred_single)


#### Test 4.3 — Edge case: all-zero features X=0

In [ ]:
# Test 4.3: all feature values are zero

model_feat_zero = make_cart_model()

X_zeros = np.zeros_like(X_toy)
model_feat_zero.train(X_zeros, y_toy)

y_pred_zeros = model_feat_zero.predict(X_zeros)
loss_zeros = model_feat_zero.loss(X_zeros, y_toy)

assert y_pred_zeros.shape == y_toy.shape
assert np.isfinite(loss_zeros)

print("Test 4.3 passed: all-zero features edge case handled correctly.")
print("Predicted labels:", y_pred_zeros)
print("Loss on all-zero features:", loss_zeros)


### Test 5: Reproduce sklearn’s DecisionTreeClassifier 

[这个部分感恩节前感觉可能不需要!]

In [6]:
# Sklearn CART (using Gini impurity)

sk_cart = DecisionTreeClassifier(
    criterion="gini",
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=0
)
sk_cart.fit(X_train, y_train)

y_pred_sk = sk_cart.predict(X_test)
sk_acc = accuracy_score(y_test, y_pred_sk)

print(f"Sklearn CART test accuracy: {sk_acc:.3f}")


Sklearn CART test accuracy: 0.912


In [ ]:
# Our own CART implementation

my_cart = make_cart_model()
my_cart.train(X_train, y_train)

y_pred_my = my_cart.predict(X_test)
my_acc = accuracy_score(y_test, y_pred_my)

print(f"Our CART test accuracy: {my_acc:.3f}")


In [ ]:
# Test 5: Compare our predictions with sklearn's CART classifier

same_predictions = np.array_equal(y_pred_sk, y_pred_my)
acc_diff = abs(sk_acc - my_acc)

print("Same predictions as sklearn?", same_predictions)
print(f"Absolute accuracy difference: {acc_diff:.6f}")

# Depending on implementation details, exact matching or near-matching are acceptable.
assert acc_diff < 1e-6 or same_predictions, \
    "Our CART implementation should match sklearn's behavior (up to small numerical or tie-breaking differences)."

print("Test 5 passed: our CART implementation successfully reproduces sklearn performance.")
